# CS 6290: Privacy-enhancing Technologies
## Assignment 2

(Acknowledgement: this assignment is adapted from [Homework 7 for UVM CS 3110, developed by Joseph P. Near](https://github.com/jnear/cs3110-data-privacy/blob/main/homework/CS3110_HW_7.ipynb).)

In [1]:
# Install dependencies (Suppose you are using Python3)
import sys
import platform

# Detect OS and adjust pip install command
if platform.system() in ["Linux", "Darwin"]:  # Darwin is macOS
    pip_cmd = f"{sys.executable} -m pip install --break-system-packages"
else:  # Windows or others
    pip_cmd = f"{sys.executable} -m pip install"

# Install dependencies
!{pip_cmd} pandas
!{pip_cmd} numpy
!{pip_cmd} scipy
!{pip_cmd} matplotlib

In [2]:
# Load the data and libraries
import pandas as pd
import numpy as np
import random
from scipy import stats
import matplotlib.pyplot as plt

def laplace_mech(v, sensitivity, epsilon):
    return v + np.random.laplace(loc=0, scale=sensitivity / epsilon)

def laplace_mech_vec(vec, sensitivity, epsilon):
    return [v + np.random.laplace(loc=0, scale=sensitivity / epsilon) for v in vec]

def gaussian_mech(v, sensitivity, epsilon, delta):
    return v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)

def gaussian_mech_vec(vec, sensitivity, epsilon, delta):
    return [v + np.random.normal(loc=0, scale=sensitivity * np.sqrt(2*np.log(1.25/delta)) / epsilon)
            for v in vec]

def pct_error(orig, priv):
    return np.abs(orig - priv)/orig * 100.0

adult = pd.read_csv('./adult_with_pii.csv')

In [3]:
adult

,Name,DOB,SSN,Zip,Age,Workclass,fnlwgt,Education,Education-Num,Marital Status,Occupation,Relationship,Race,Sex,Capital Gain,Capital Loss,Hours per week,Country,Target
0,Karrie Trusslove,9/7/1967,732-14-6110,64152,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,Brandise Tripony,6/7/1988,150-19-2766,61523,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,Brenn McNeely,8/6/1991,725-59-9860,95668,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,Dorry Poter,4/6/2009,659-57-4974,25503,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,Dick Honnan,9/16/1951,220-93-3811,75387,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32556,Ardyce Golby,10/29/1961,212-61-8338,41328,27,Private,257302,Assoc-acdm,12,Married-civ-spouse,Tech-support,Wife,White,Female,0,0,38,United-States,<=50K
32557,Jean O'Connor,6/28/1952,737-32-2919,94735,40,Private,154374,HS-grad,9,Married-civ-spouse,Machine-op-inspct,Husband,White,Male,0,0,40,United-States,>50K
32558,Reuben Skrzynski,8/9/1966,314-48-0219,49628,58,Private,151910,HS-grad,9,Widowed,Adm-clerical,Unmarried,White,Female,0,0,40,United-States,<=50K
32559,Caye Biddle,5/19/1978,647-75-3550,8213,22,Private,201490,HS-grad,9,Never-married,Adm-clerical,Own-child,White,Male,0,0,20,United-States,<=50K



## Prerequisites

- Before diving into the programming, we suggest reviewing the following resources to refresh or build your understanding:
  - [Differential Privacy](https://programming-dp.com/ch3.html) – Revisit the basics of differential privacy and the Laplace Mechanism.
  - [Properties of Differential Privacy](https://programming-dp.com/ch4.html) – Learn about Sequential Composition in DP.
  - [The Gaussian Mechanism](https://programming-dp.com/ch6.html#the-gaussian-mechanism) – Explore the Gaussian Mechanism.
  - [Variants of Differential Privacy](https://programming-dp.com/ch8.html) – Understand Rényi DP.

- For those eager to deepen their understanding of the theoretical foundations of differential privacy, consider reading:
  - [Chapters 2 & 3 of *The Algorithmic Foundations of Differential Privacy*](https://www.cis.upenn.edu/~aaroth/Papers/privacybook.pdf) by Cynthia Dwork and Aaron Roth.
  
- We assume you have basic knowledge about `numpy` and `pandas`. For those unfamiliar with these libraries, please visit the crash course provided by Kaggle https://www.kaggle.com/learn/pandas.

## Range Queries

A *range query* counts the number of rows in the dataset which have a value lying in a given range. For example, "how many participants are between the ages of 21 and 33?" is a range query. A *workload* of range queries is just a list of range queries. The code below generates 100 random range queries over ages in the adult dataset.

In [4]:
def range_query(df, col, a, b):
    return len(df[(df[col] >= a) & (df[col] < b)])

random_lower_bounds = [random.randint(1, 70) for _ in range(100)]
random_workload = [(lb, random.randint(lb, 100)) for lb in random_lower_bounds]
real_answers = [range_query(adult, 'Age', lb, ub) for (lb, ub) in random_workload]
print('First 5 queries: ', random_workload[:5])

First 5 queries:  [(3, 99), (28, 52), (26, 28), (67, 86), (59, 99)]


## Question 1-1 (1 point)

Write code to answer a workload of range queries using `laplace_mech` and sequential composition. Your solution should have a **total privacy cost of epsilon**.

In [5]:
def workload_laplace(workload, epsilon):
    # YOUR CODE HERE
    
    raise NotImplementedError()

print('First 4 answers:', workload_laplace(random_workload, 1.0)[:4])

NotImplementedError: 

In [ ]:
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers, workload_laplace(random_workload, 1.0))]
print('Average absolute error:', np.mean(errors))
assert np.mean(errors) > 50
assert np.mean(errors) < 200

## Question 1-2 (1 point)

Write code to answer a workload using `laplace_mech_vec` - the version of the Laplace mechanism for **vector-valued** queries. Your solution should *not* use sequential composition, and should have a total privacy cost of `epsilon`.

*Hint*: remember to use L1 global sensitivity.

In [ ]:
def workload_laplace_vec(workload, epsilon):
    # YOUR CODE HERE
    raise NotImplementedError()

print('First 4 answers:', workload_laplace_vec(random_workload, 1.0)[:4])

In [ ]:
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers, workload_laplace_vec(random_workload, 1.0))]
print('Average absolute error:', np.mean(errors))
assert np.mean(errors) > 50
assert np.mean(errors) < 200

## Question 1-3 (1 point)

In 2-5 sentences, answer the following:
- Did the two solutions differ in terms of their accuracy?
- How do they differ in terms of their use of composition properties of differential privacy?

YOUR ANSWER HERE

## Question 2-1 (1 point)

Write code to answer a workload using `gaussian_mech_vec` - the version of the Gaussian mechanism for vector-valued queries. Your solution should not use sequential composition, should satisfy $(\epsilon, \delta)$-differential privacy, and should have a total privacy cost of (`epsilon`, `delta`).

*Hint*: remember to use L2 sensitivity.

In [ ]:
def workload_gaussian_vec(workload, epsilon, delta):
    # YOUR CODE HERE
    raise NotImplementedError()

print('First 4 answers:', workload_gaussian_vec(random_workload, 1.0, 1e-5)[:4])

In [ ]:
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers, workload_gaussian_vec(random_workload, 1.0, 1e-5))]
print('Average absolute error:', np.mean(errors))
assert np.mean(errors) > 10
assert np.mean(errors) < 100

## Question 2-2 (1 point)

In 2-5 sentences, answer the following:
- Of your solutions in Q1-1, Q1-2, and Q2-1, which ones rely on *sequential composition*?
- Which solution offers the best accuracy?
- Why does this particular solution yield the best accuracy?

YOUR ANSWER HERE

## Question 3-1 (2 points)

Re-implement your solution to Question 2 using *Rényi differential privacy*. Your solution should satisfy $(\alpha, \bar\epsilon)$-RDP.

*Hint*: See https://programming-dp.com/ch8.html for more about *Rényi DP*.

In [ ]:
def workload_gaussian_vec_RDP(workload, alpha, epsilon_bar):
    # YOUR CODE HERE
    raise NotImplementedError()

print('First 4 answers:', workload_gaussian_vec(random_workload, 1.0, 1e-5)[:4])

In [ ]:
# TEST CASE
errors = [abs(r_a - l_a) for (r_a, l_a) in zip(real_answers, workload_gaussian_vec_RDP(random_workload, 5, 0.1))]
print('Average absolute error:', np.mean(errors))
assert np.mean(errors) > 10
assert np.mean(errors) < 100

## Question 3-2 (1 point)

Implement a function `convert_RDP_ED` to convert from the $(\alpha, \bar\epsilon)$ of Rényi differential privacy to the $(\epsilon, \delta)$ of approximate differential privacy. Your function should also take the desired value of $\delta$.

In [ ]:
def convert_RDP_ED(alpha, epsilon_bar, delta):
    # YOUR CODE HERE
    raise NotImplementedError()

convert_RDP_ED(5, 0.1, 1e-5)

In [ ]:
# TEST CASE
assert convert_RDP_ED(5, 0.1, 1e-5) == 2.9782313662425572
assert convert_RDP_ED(40, 0.1, 1e-5) == 0.39520321705051864
assert convert_RDP_ED(500, 1.0, 1e-5) == 1.02307199491978
assert convert_RDP_ED(40, 1.0, 1e-5) == 1.2952032170505188

## Question 3-3 (2 points)

In 2-5 sentences, answer the following:
- Try various values for `alpha` and `epsilon_bar` in `convert_RDP_ED`. At what values do you observe an $(\epsilon, \delta)$ value around $(1.0, 10^{-5})$?
- Try these values for `alpha` and `epsilon_bar` in `workload_gaussian_vec_RDP`. How does the error compare to using `workload_gaussian_vec`?
- Is it useful to use Rényi differential privacy to answer workloads of range queries? Or is regular $(\epsilon, \delta)$-differential privacy just as good?

YOUR ANSWER HERE